In [46]:
from transformers import (
    DataCollator,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)
from primeqa.qg.processors.data_loader import QGDataLoader
import torch
from dataclasses import dataclass,field
from primeqa.qg.models.qg_model import QGModel
from primeqa.qg.trainers.qg_trainer import QGTrainer
from typing import Optional, List, Dict
from primeqa.qg.trainers.qg_trainer_utils import T2TDataCollator, ModelArguments, DataTrainingArguments, QGTrainingArguments, InferenceArguments


import json
import logging
import os

print("Done importing")

Done importing


In [39]:
    model_name_or_path="t5-base"
    modality="table"
    dataset_name="wikisql"
    max_len=200
    target_max_len=40
    output_dir="models/qg/trials"
    learning_rate=0.0001
    num_train_epochs=4
    per_device_train_batch_size=32
    print("done2")

    training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=False,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    evaluation_strategy='no',
    learning_rate=learning_rate,
    seed=10,
    )
    print(training_args)
    print("done3")


    model_args = ModelArguments(
        model_name_or_path=model_name_or_path,
        modality=modality
    )

    print(model_args)
    print("done4")

    data_args = DataTrainingArguments(
        dataset_name = dataset_name,
        max_len = max_len,
        target_max_len = target_max_len
    )

    print(data_args)
    print("done5")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


done2
TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=0.0001,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=models/qg/trials

In [14]:
tqg = QGModel(model_args.model_name_or_path, modality=model_args.modality)
model = tqg.model
tokenizer = tqg.tokenizer


In [40]:
data_args = DataTrainingArguments(
        dataset_name = dataset_name,
        max_len = max_len,
        target_max_len = target_max_len
)

qgdl = QGDataLoader(tokenizer,data_args)

train_dataset = qgdl.create("train")

valid_dataset = qgdl.create("validation")


Using custom data configuration default
Reusing dataset wiki_sql (/u/jaydesen/.cache/huggingface/datasets/wiki_sql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)
100%|██████████| 56355/56355 [00:26<00:00, 2113.10it/s]


  0%|          | 0/57 [00:00<?, ?ba/s]

/dccstor/cssblr/jaydeep/1qa_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Using custom data configuration default
Reusing dataset wiki_sql (/u/jaydesen/.cache/huggingface/datasets/wiki_sql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)
100%|██████████| 8421/8421 [00:03<00:00, 2198.25it/s]


  0%|          | 0/9 [00:00<?, ?ba/s]

In [41]:
print(train_dataset)
print(train_dataset[0])

Dataset({
    features: ['question', 'input', 'input_ids', 'attention_mask', 'target_ids', 'target_attention_mask'],
    num_rows: 56355
})
{'input_ids': tensor([ 1738, 32100,  2507,     7, 32100, 12892, 22031, 32101,  4081, 32101,
          180,  9744,   566,     3,  6727, 13733, 24933,   188, 32102,   150,
        22031,    30,   750,   939, 32103,  1015,    87,    17, 21301, 10972,
        32104,  5027,    87,  1549,  9232,  3243, 32104, 12439, 32104, 12892,
        22031, 32104, 12892,   939, 32104,  2507,     7,     1,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,   

done6


In [42]:
trainer = QGTrainer(
            model=model,
            tokenizer = tokenizer,
            args=training_args,
            train_dataset=train_dataset,
            valid_dataset=valid_dataset,
            data_collator=T2TDataCollator()
        )

print("done8")



done8


In [47]:
  print(model_args.model_name_or_path)
  trainer.train(
        model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None, resume_from_checkpoint=None
        )
  print("done9")

t5-base


NameError: name 'checkpoint' is not defined